# Model Training

Train LSTM-based models and evaluate with MSE, R^2, and predicted degredation.

In [47]:
import sys
sys.path.append('/content/drive/MyDrive/msc_data_analytics_thesis_project_pose_estimation')

In [48]:
# ✅ Environment Setup
from utils.setup import setup_environment
base_path = setup_environment(mount_gdrive=True)

import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from tensorflow.keras.layers import LSTM, Bidirectional, GRU, SimpleRNN, Dense, Dropout, Input, BatchNormalization, Conv1D

from utils.config import ENGINEERED_DIR, PROCESSED_DIR, EXERCISES, RESULTS_DIR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Environment set. Project base path: /content/drive/MyDrive/pose-estimation-research


In [ ]:
def save_individual_plots(history, y_true, y_pred, params, exercise_dir, sample_size=50):
    """Save each visualization as separate image file"""
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    errors = y_true - y_pred
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # 1. Loss curves plot
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title(f"{params['exercise']} ({params['features']}) - Training & Validation Loss")
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    loss_path = os.path.join(exercise_dir, f"{params['exercise']}_{params['features']}_loss_curves.png")
    plt.savefig(loss_path)
    plt.close()

    # 2. Prediction scatter plot
    plt.figure(figsize=(8, 5))
    plt.scatter(y_true, y_pred, alpha=0.3)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'k--')
    plt.title(f"{params['exercise']} ({params['features']})\nPredictions vs Truth (MAE: {mae:.4f}, R2: {r2:.4f})")
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.grid(True)
    scatter_path = os.path.join(exercise_dir, f"{params['exercise']}_{params['features']}_scatter.png")
    plt.savefig(scatter_path)
    plt.close()

    # 3. Error distribution plot
    plt.figure(figsize=(8, 5))
    sns.histplot(errors, kde=True)
    plt.title(f"{params['exercise']} ({params['features']}) - Error Distribution")
    plt.xlabel('Prediction Error')
    plt.grid(True)
    error_path = os.path.join(exercise_dir, f"{params['exercise']}_{params['features']}_errors.png")
    plt.savefig(error_path)
    plt.close()

    # 4. Line plot comparison
    plt.figure(figsize=(12, 5))
    sample_size = min(sample_size, len(y_true))
    plt.plot(y_true[:sample_size], label="True", marker='o', markersize=3)
    plt.plot(y_pred[:sample_size], label="Predicted", alpha=0.7, linestyle='--', marker='x', markersize=3)
    plt.title(f"{params['exercise']} ({params['features']}) - First {sample_size} Samples Comparison")
    plt.xlabel("Sample index")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    line_path = os.path.join(exercise_dir, f"{params['exercise']}_{params['features']}_line_comparison.png")
    plt.savefig(line_path)
    plt.close()

    return {
        'loss_plot': loss_path,
        'scatter_plot': scatter_path,
        'error_plot': error_path,
        'line_plot': line_path
    }

In [ ]:
def save_enhanced_results(history, y_true, y_pred, params, exercise_dir):
    """Save all results including individual plots"""
    # Save individual plots
    plot_paths = save_individual_plots(history, y_true, y_pred, params, exercise_dir)

    # Calculate metrics
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    metrics = {
        'final_train_loss': history.history['loss'][-1],
        'final_val_loss': history.history['val_loss'][-1],
        'final_train_mae': history.history['mae'][-1],
        'final_val_mae': history.history['val_mae'][-1],
        'prediction_mae': mean_absolute_error(y_true, y_pred),
        'prediction_r2': r2_score(y_true, y_pred),
        'num_epochs_run': len(history.history['loss']),
        'best_epoch': np.argmin(history.history['val_loss']) + 1,
        **plot_paths  # Add paths to the individual plots
    }

    # Combine parameters and metrics
    results = {**params, **metrics}

    # Save as CSV
    csv_filename = f"{params['exercise']}_{params['features']}_metrics.csv"
    csv_path = os.path.join(exercise_dir, csv_filename)
    pd.DataFrame([results]).to_csv(csv_path, index=False)

    return results

In [ ]:
def load_and_preprocess_data(exercise: str, augment_with_features: bool = True):
    """Load and preprocess the data with proper validation"""
    dir_path = os.path.join(ENGINEERED_DIR, exercise)

    # Determine which feature file to load
    feature_file = "X_featured.npy" if augment_with_features else "X.npy"
    feature_path = os.path.join(dir_path, feature_file)
    target_path = os.path.join(dir_path, "y.npy")

    # Validate files exist
    if not os.path.exists(feature_path):
        raise FileNotFoundError(f"Feature file not found: {feature_path}")
    if not os.path.exists(target_path):
        raise FileNotFoundError(f"Target file not found: {target_path}")

    try:
        X = np.load(feature_path)
        y = np.load(target_path)

        # Validate shapes
        if len(X) != len(y):
            raise ValueError(f"X and y have different lengths: {len(X)} vs {len(y)}")

        if X.ndim != 3:  # Should be (samples, timesteps, features)
            raise ValueError(f"X should be 3D array, got {X.ndim}D")

        # Robust scaling for features
        original_shape = X.shape
        scaler_x = RobustScaler()
        X = scaler_x.fit_transform(X.reshape(-1, X.shape[-1])).reshape(original_shape)

        # Scale targets to [0,1] with epsilon to avoid division by zero
        y_min, y_max = y.min(), y.max()
        y_range = y_max - y_min
        if y_range < 1e-10:  # Handle near-constant y
            y_range = 1.0
        y = (y - y_min) / y_range

        return X, y

    except Exception as e:
        raise RuntimeError(f"Error loading {exercise} data: {str(e)}")

In [ ]:
def build_enhanced_rnn(input_shape):
    """Build model with proper output shape"""
    inputs = Input(shape=input_shape)
    conv = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Dropout(0.2)(conv)

    gru = GRU(128, return_sequences=False)(conv)
    gru = BatchNormalization()(gru)
    gru = Dropout(0.3)(gru)

    dense = Dense(128, activation='relu')(gru)
    dense = Dropout(0.3)(dense)
    output = Dense(1)(dense)

    return Model(inputs=inputs, outputs=output)

In [ ]:
def train_enhanced_model(X, y, exercise_name, augment_flag):
    """Training with proper validation set handling"""
    # Create exercise-specific results directory
    feature_type = "featured" if augment_flag else "basic"
    exercise_dir = os.path.join(RESULTS_DIR, exercise_name, feature_type)
    os.makedirs(exercise_dir, exist_ok=True)

    # Stratified split
    y_bins = np.digitize(y, bins=np.quantile(y, [0.25, 0.5, 0.75]))
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y_bins, random_state=42
    )

    model = build_enhanced_rnn(X.shape[1:])

    # Experiment parameters
    params = {
        'exercise': exercise_name,
        'features': feature_type,
        'learning_rate': 0.005,
        'loss': 'huber',
        'batch_size': 32,
        'epochs': 100,
        'dropout_rates': [0.2, 0.3, 0.3],
        'architecture': 'Conv1D(64)-GRU(128)-Dense(128)',
        'early_stopping_patience': 20,
        'reduce_lr_patience': 10,
        'input_shape': X.shape[1:]
    }

    model.compile(
        optimizer=Adam(learning_rate=params['learning_rate']),
        loss=params['loss'],
        metrics=['mae']
    )

    callbacks = [
        EarlyStopping(patience=params['early_stopping_patience'], restore_best_weights=True),
        ReduceLROnPlateau(factor=0.5, patience=params['reduce_lr_patience'], min_lr=1e-6)
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        callbacks=callbacks,
        verbose=1
    )

    return model, history, X_val, y_val, params

In [ ]:
def run_experiment(exercise, augment_flag):
    """Run full experiment for one exercise with specified feature type"""
    try:
        feature_type = "featured" if augment_flag else "basic"
        print(f"\n{'='*50}")
        print(f"Starting experiment for: {exercise} ({feature_type} features)")
        print(f"{'='*50}")

        # Load data
        X, y = load_and_preprocess_data(exercise, augment_with_features=augment_flag)

        # Train model
        model, history, X_val, y_val, params = train_enhanced_model(
            X, y, exercise, augment_flag
        )

        # Predict and evaluate
        y_pred = model.predict(X_val)

        # Debug shapes
        print(f"Shapes - y_val: {y_val.shape}, y_pred: {y_pred.shape}")

        # Flatten arrays
        y_val_flat = y_val.flatten()
        y_pred_flat = y_pred.flatten()

        # Save results
        exercise_dir = os.path.join(RESULTS_DIR, exercise, feature_type)
        results = save_enhanced_results(
            history, y_val_flat, y_pred_flat, params, exercise_dir
        )

        # Print summary
        print(f"\nCompleted experiment for: {exercise} ({feature_type} features)")
        print(f"Final Validation MAE: {results['prediction_mae']:.4f}")
        print(f"Final Validation R2: {results['prediction_r2']:.4f}")
        print(f"Results saved to: {exercise_dir}")

        return results

    except Exception as e:
        print(f"Error occurred for {exercise} ({'featured' if augment_flag else 'basic'}): {str(e)}")
        if 'y_pred' in locals():
            print(f"y_pred shape: {y_pred.shape}")
        if 'y_val' in locals():
            print(f"y_val shape: {y_val.shape}")
        return None

In [ ]:
def generate_comparison_tables(all_results):
    """Generate comparison tables and visualizations"""
    if not all_results:
        return

    df = pd.DataFrame(all_results)

    # Save full results
    full_results_path = os.path.join(RESULTS_DIR, "all_experiments_full_results.csv")
    df.to_csv(full_results_path, index=False)

    # Create comparison tables
    comparison_dir = os.path.join(RESULTS_DIR, "comparisons")
    os.makedirs(comparison_dir, exist_ok=True)

    # 1. Basic metrics comparison
    metric_cols = ['exercise', 'features', 'prediction_mae', 'prediction_r2',
                  'num_epochs_run', 'final_val_loss']
    metrics_df = df[metric_cols].sort_values(['exercise', 'features'])
    metrics_path = os.path.join(comparison_dir, "key_metrics_comparison.csv")
    metrics_df.to_csv(metrics_path, index=False)

    # 2. Feature augmentation impact
    feat_impact = df.pivot_table(
        index='exercise',
        columns='features',
        values=['prediction_mae', 'prediction_r2'],
        aggfunc='mean'
    )
    feat_impact_path = os.path.join(comparison_dir, "feature_augmentation_impact.csv")
    feat_impact.to_csv(feat_impact_path)

    # 3. Create visual comparison plots
    plt.figure(figsize=(12, 6))

    # MAE comparison
    plt.subplot(1, 2, 1)
    sns.barplot(data=df, x='exercise', y='prediction_mae', hue='features')
    plt.title('MAE Comparison by Exercise and Features')
    plt.ylabel('MAE')
    plt.xticks(rotation=45)

    # R2 comparison
    plt.subplot(1, 2, 2)
    sns.barplot(data=df, x='exercise', y='prediction_r2', hue='features')
    plt.title('R2 Comparison by Exercise and Features')
    plt.ylabel('R2 Score')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plot_path = os.path.join(comparison_dir, "metrics_comparison.png")
    plt.savefig(plot_path)
    plt.close()

    print(f"\nComparison tables and plots saved to: {comparison_dir}")

In [ ]:
def main():
    """Run experiments for all exercises and feature types"""
    all_results = []

    for exercise in EXERCISES:
        # Run with basic features
        basic_results = run_experiment(exercise, augment_flag=False)
        if basic_results:
            all_results.append(basic_results)

        # Run with augmented features
        featured_results = run_experiment(exercise, augment_flag=True)
        if featured_results:
            all_results.append(featured_results)

    # Generate comparison tables and plots
    generate_comparison_tables(all_results)

    # Print final summary
    if all_results:
        summary_df = pd.DataFrame(all_results)
        print("\nExperiment Summary:")
        print(summary_df[['exercise', 'features', 'prediction_mae', 'prediction_r2',
                         'num_epochs_run', 'final_val_loss']].to_string(index=False))
    else:
        print("\nNo experiments completed successfully.")

In [52]:
main()


Starting experiment for: squat (basic features)
Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.6424 - mae: 1.0415 - val_loss: 0.0723 - val_mae: 0.3155 - learning_rate: 0.0050
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.2267 - mae: 0.5485 - val_loss: 0.0587 - val_mae: 0.2848 - learning_rate: 0.0050
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0719 - mae: 0.2998 - val_loss: 0.0475 - val_mae: 0.2524 - learning_rate: 0.0050
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0496 - mae: 0.2516 - val_loss: 0.0547 - val_mae: 0.2730 - learning_rate: 0.0050
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0324 - mae: 0.1988 - val_loss: 0.0403 - val_mae: 0.2314 - learning_rate: 0.0050
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0272 - mae: 0.1830 - val_loss: 0.0308 - val_mae: 0.2016 - learning_rate: 0.0050
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0240 - mae: 0.1722 - val_loss: 0.0288